In [1]:
#package install
!pip install tensorflow==2.8.4 tensorflow-gpu==2.8.4 opencv-python mediapipe
!pip install mediapipe
!pip install matplotlib
!pip install scikit-learn
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
!unzip "features.zip"

In [10]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/content/content/MP_Data')

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou','done','ok','learn'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'done': 3, 'ok': 4, 'learn': 5}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(360, 30, 1662)

In [14]:
np.array(labels).shape

(360,)

In [15]:
X = np.array(sequences)

In [16]:
X.shape

(360, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
X_train.shape

(288, 30, 1662)

In [22]:
X_test.shape

(72, 30, 1662)

**LSTM MODEL**

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
9/9 [==============================] - 5s 214ms/step - loss: 1.8142 - categorical_accuracy: 0.1528
Epoch 2/2000
9/9 [==============================] - 1s 150ms/step - loss: 1.7322 - categorical_accuracy: 0.2465
Epoch 3/2000
9/9 [==============================] - 1s 120ms/step - loss: 1.4258 - categorical_accuracy: 0.3507
Epoch 4/2000
9/9 [==============================] - 1s 121ms/step - loss: 1.2777 - categorical_accuracy: 0.4132
Epoch 5/2000
9/9 [==============================] - 1s 121ms/step - loss: 1.2147 - categorical_accuracy: 0.4271
Epoch 6/2000
9/9 [==============================] - 1s 120ms/step - loss: 1.1304 - categorical_accuracy: 0.4375
Epoch 7/2000
9/9 [==============================] - 1s 128ms/step - loss: 1.1549 - categorical_accuracy: 0.4653
Epoch 8/2000
9/9 [==============================] - 1s 115ms/step - loss: 2.0678 - categorical_accuracy: 0.4340
Epoch 9/2000
9/9 [==============================] - 1s 118ms/step - loss: 1.3206 - categorical_accuracy:

KeyboardInterrupt: 

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-trai

In [29]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[0])]

'iloveyou'

In [31]:
actions[np.argmax(y_test[0])]

'iloveyou'

**Save Weights**

In [32]:
model.save('action.h5')

**Evaluation**

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [34]:
yhat = model.predict(X_test)

In [35]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [36]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[52,  3],
        [ 7, 10]],

       [[57,  1],
        [ 2, 12]],

       [[59,  6],
        [ 0,  7]],

       [[63,  0],
        [ 0,  9]],

       [[56,  0],
        [ 1, 15]],

       [[63,  0],
        [ 0,  9]]])

In [37]:
accuracy_score(ytrue, yhat)

0.8611111111111112